   i.      CAB-LA Patient IDI cohort: Per Table 1 (below), we will systematically select ~n=30 patients per each region (South West Uganda and Kenya) (total for both countries 40 < ~n < 72; due to time and resource limitations, a smaller sample may be collected during this study), with samples within region to be balanced proportional to overall intervention enrolment by:

    Original trials (OPD, ANC/PNC, VHT);
    Gender; and
    Community.
Please note, we would to interview some of those who opted for the injection but then after receiving one or two injections have decided not to continue with them. However, we would not want to interview more than a total of 10 - 15 of those no longer taking CAB-LA. Please indicate those persons in the list (perhaps with an *).

Table 1 (below)
·  N= 12-20 outpatient clinic clients (n=3-5 per 4 communities)

 

·  N= 16-20 Antenatal and postnatal clients (n= 2-3 per antenatal and postnatal group per 4 communities)

 

·  N= 12-20 men and women recruited from VHT clients (n= 3-5 per 4 communities, gender-balanced)


In [38]:
# import the required libraries
import pandas as pd
import numpy as np
import pyodbc
import json
import sqlalchemy
from sqlalchemy import create_engine
import datetime;
import os

In [2]:
# Create DB connection
def connect_to_mysql():
    # Load connection details from JSON file
    with open('connection_details.json', 'r') as file:
        connection_details = json.load(file)
        mysql_credentials=connection_details['mysql']
    
    # Establish connection to MySQL database
    try:
        connection = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                .format(host=mysql_credentials['host'],
                user=mysql_credentials['user'],
                pw=mysql_credentials['password'],
                db=mysql_credentials['cab_database']))
        
        print("Successfully connected to MySQL database")
        return connection        
        
        # Close the database connection
        connection.close()
        print("Connection closed")
        return df_partcipant
    except:
        print("Failed to connect to MySQL database: {}")
        return false

In [98]:
#Get connection to the db
conn = connect_to_mysql()

# Pull data
sql = """
    SELECT
        *
    FROM        
        d_participant;
        """
        
df = pd.read_sql(sql, conn)
df = df[['subjid', 'participant_id', 'trial', 'clinic',
       'country', 'study_arm', 'screened', 'screened_met', 'consented',
       'enrolled', 'baseline_visit', 'cab_screen_date', 'cab_enr_date', 'cab_baseline_date', 'age',
       'age_group', 'sex']]



# get List of participants who have had a cab injection
sql = """
    SELECT
        subjid, vdate,cab_vweek, screen_cab_met, injection_received
    FROM 
        schedc_cab_baseline
    WHERE
       screen_cab_met = 1 AND injection_received = 1; 
    """
df_cab = pd.read_sql(sql, conn)

sql = """
    SELECT
        subjid, vdate,cab_vweek, screen_cab_met, injection_received, stop_cab
    FROM 
        schedc_cab_followup; 
    """
df_cab_fu = pd.read_sql(sql, conn)
df_cab_fu['vdate'] = pd.to_datetime(df_cab_fu['vdate'])
df_cab_fu.sort_values('vdate', inplace=True, ascending=False)
df_cab_fu.loc[df_cab_fu['screen_cab_met'] == 0, 'stop_cab'] = 1

df_latest_cab_status = df_cab_fu.groupby('subjid').first().reset_index()

# get those on who ever started cab only
df_int = df[(df['study_arm'] == 'Int') & (df['enrolled'] == 1)]
df_int = df_int.merge(df_cab, on = 'subjid', how = 'inner')
df_int = df_int.merge(df_latest_cab_status, on = 'subjid', how = 'left')
print(df_int.shape)

# get individual trials
df_anc = df_int[(df_int['trial'] == 'ANC')]
df_opd = df_int[(df_int['trial'] == 'OPD')]
df_vht = df_int[(df_int['trial'] == 'VHT/CHV')]

# Get Sample some of those who opted for the injection but then after receiving one or two injections have decided not to continue with them.
# Get those who are wk 24 since start of cab injection and have stopped injection

# get max injections given
summary_data = df_cab_fu.groupby(['subjid']).agg({'injection_received':[
    ('injections', lambda x: np.where(x == 1, 1,0).sum()),
    ('cab_vweek',lambda x: np.where(df_cab_fu['cab_vweek'] == 99, 0,df_cab_fu['cab_vweek']).max()),
    ('stop_cab_vweek',lambda x: np.where(df_cab_fu['stop_cab'] == 1.0, df_cab_fu['cab_vweek'],99).min())
]})
# Reset the index to make  regular columns
summary_data = summary_data.reset_index()

# Rename the aggregated column for clarity
summary_data.columns = ['subjid', 'n_injections', 'max_vweek','stop_cab_vweek']


# add the baseline injection
summary_data['n_injections_inc_baseline'] = summary_data['n_injections'] + 1
# expected injections as at wk 24
summary_data['expected_injections'] = 5
df_stopped = df_cab_fu[df_cab_fu['stop_cab'] == 1]
df_stopped_cab = summary_data[(summary_data['n_injections_inc_baseline']<=2) & (summary_data['subjid'].isin(df_stopped['subjid']))]

df_stopped_cab = df_stopped_cab.merge(df_int[['subjid','trial','sex', 'clinic']], on = 'subjid', how = 'inner')

# Get the sample
# Define the stratification variables for OPD and VHT
strata_columns = ['trial', 'clinic', 'sex']

# Define the desired sample size
sample_size = 1  # Adjust this according to your requirements

# Perform stratified sampling
stratified_sample = df_stopped_cab.groupby(strata_columns, as_index=False, group_keys=False).apply(lambda x: x.sample(n=sample_size, random_state=42))
stratified_sample['ever_stopped_cab'] = 1
# drop anyone who had stopped cab
df_anc = df_anc[~df_anc['subjid'].isin(df_stopped_cab['subjid'])]
df_opd = df_opd[~df_opd['subjid'].isin(df_stopped_cab['subjid'])]
df_vht = df_vht[~df_vht['subjid'].isin(df_stopped_cab['subjid'])]

# Define the desired sample size
sample_size = 5  # Adjust this according to your requirements

# Perform stratified sampling
#stratified_sample = df_int.groupby(strata_columns, as_index=False, group_keys=False).apply(lambda x: x.sample(n=sample_size, random_state=42))

sample_size = 4
stratified_sample_anc = df_anc.groupby(['clinic'], as_index=False, group_keys=False).apply(lambda x: x.sample(n=sample_size, random_state=42))

sample_size = 2

stratified_sample_opd = df_opd.groupby(['clinic', 'sex'], as_index=False, group_keys=False).apply(lambda x: x.sample(n=sample_size, random_state=42))
stratified_sample_vht = df_vht.groupby(['clinic', 'sex'], as_index=False, group_keys=False).apply(lambda x: x.sample(n=sample_size, random_state=42))

stratified_sample_anc['ever_stopped_cab'] = 0
stratified_sample_opd['ever_stopped_cab'] = 0
stratified_sample_vht['ever_stopped_cab'] = 0


include_vars = ['subjid','trial','clinic','sex','ever_stopped_cab']

# Concatenate the DataFrames vertically (bind rows)
df_out = pd.concat([stratified_sample_anc[include_vars], stratified_sample_opd[include_vars], stratified_sample_vht[include_vars], stratified_sample[include_vars]])


# Reset the index
df_out = df_out.reset_index(drop=True)

df_out.sort_values('clinic', inplace=True, ascending=False)
df_out

# check if in ANC trial the the 3 participants were included
#Postnatal vs antenatal (Anyone who signed pregnancy consent will be classified as antenatal)
# There are only 3 Pregnancies reported in ANC Trial
anc_antenatal = ['SP11005006', 'SP11007006', 'SP11004005']
print('ANC LIST')
df_out[df_out['subjid'].isin(anc_antenatal)]

# print line list
df_out.to_csv('qual_IDI_list.csv', encoding='utf-8', index=False)

# Sample Providers
sql = """
    SELECT
        *
    FROM 
        prvdr_mm_survey; 
    """
df_prvdr = pd.read_sql(sql, conn)

#TODO : get list from clinic to help in classifying


Successfully connected to MySQL database
(265, 26)
(48, 9)
(15, 10)
(16, 27)
(16, 27)
(12, 27)
ANC LIST


,starttime,subjid,subjid2,vdate,study,study_visit,provider_consent,no_consent_reason,prior_know_cabla,colleagues_know_cabla,...,barriers_anticipate_cabla_other,barriers_faced_cabla,barriers_faced_cabla_other,provider_id,provider_initials,comments,swver,stoptime,run_uuid,site
0,2023-02-07 17:24:33,26,26,2023-02-07,0,0,1,-9,3,3,...,-9,-9,-9,26,None,CAB-LA IS GENERALLY PERCEIVED AS A GOOD OPTION...,SEARCH SAPPHIRE CAB-LA ver.2023.01.06,'2023-02-07 17:36:38',0d911961-87c1-4d55-a64c-53655abca748,None
1,2023-01-17 12:33:08,76,76,2023-01-17,6,0,1,-9,2,2,...,-9,-9,-9,76,None,-6,SEARCH SAPPHIRE CAB-LA ver.2023.01.06,'2023-01-17 12:44:49',fffdc1c1-254c-4cb0-8bd4-aa7758800b80,None
2,2023-01-10 13:16:56,12,12,2023-01-10,2,0,1,-9,1,4,...,-9,-9,-9,12,None,-6,SEARCH SAPPHIRE CAB-LA ver.2023.01.06,'2023-01-10 13:26:14',75594d00-2a60-4238-99b1-8f4d3d54e002,None
3,2023-01-17 11:54:49,67,67,2023-01-09,7,0,1,-9,1,2,...,-9,-9,-9,67,None,-6,SEARCH SAPPHIRE CAB-LA ver.2023.01.06,'2023-01-17 12:17:08',a275d8e0-267a-44a2-b5b1-97df95c76480,None
4,2023-02-07 14:24:19,31,31,2023-02-07,0,0,1,-9,2,5,...,-9,-9,-9,31,None,-6,SEARCH SAPPHIRE CAB-LA ver.2023.01.16,'2023-02-07 14:26:15',87a2b4ce-f091-4d20-abf8-8fa5ccc385d9,None
5,2023-06-14 08:02:36,3,3,2023-06-14,3,1,1,-9,-9,-9,...,-9,-9,-9,3,None,I DID NOT EXPERIENCE ANY DIFFICULTIES WHILE AD...,SEARCH SAPPHIRE CAB-LA ver.2023.06.05,'2023-06-14 08:10:14',20300c5a-d9d4-4c53-b7c1-cf7f79a27a94,None
6,2023-01-10 11:37:22,3,3,2023-01-10,3,0,1,-9,2,2,...,-9,-9,-9,3,None,-6,SEARCH SAPPHIRE CAB-LA ver.2023.01.06,'2023-01-10 11:43:57',20300c5a-d9d4-4c53-b7c1-cf7f79a27a94,None
7,2023-06-07 12:26:29,11,11,2023-06-07,1,1,1,-9,-9,-9,...,-9,-9,-9,11,None,-6,SEARCH SAPPHIRE CAB-LA ver.2023.06.05,'2023-06-07 12:29:18',70b3dcbe-41fc-4629-a39b-0b806345ef4b,None
8,2023-01-10 12:16:01,11,11,2023-01-10,1,0,1,-9,2,2,...,-9,-9,-9,11,None,-6,SEARCH SAPPHIRE CAB-LA ver.2023.01.03,'2023-01-10 12:21:40',70b3dcbe-41fc-4629-a39b-0b806345ef4b,None
9,2023-02-08 10:54:01,17,17,2023-02-08,7,0,1,-9,3,3,...,-9,-9,-9,17,None,-6,SEARCH SAPPHIRE CAB-LA ver.2023.01.16,'2023-02-08 10:58:27',3ab517e2-8d5d-4d48-8e98-92054a00fa6c,None
